In [ ]:
import os

import sys, importlib

import logging
import warnings
import time
from copy import deepcopy
from typing import Tuple


import numpy as np
import xarray as xr
import pandas as pd

import math
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import seaborn as sns

import skimage.measure
import scipy

import multiprocessing


from utils import circular_mean, circular_variance

from generic import load_image, save_image
importlib.reload(sys.modules['generic'])



log = logging.getLogger(__name__)

import matplotlib.font_manager

from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
plt.style.context('seaborn-paper')
# plt.style.use('seaborn-notebook')
# plt.style.use('seaborn-poster')
# plt.style.use('seaborn-talk')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)


linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

    

from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
from matplotlib import font_manager
matplotlib.font_manager.findSystemFonts(fontpaths='/usr/share/fonts/truetype', fontext='ttf')
font_manager.findfont('Helvetica', fallback_to_default=False)

# Gather images

In [ ]:
# load images and segmentations
Images = dict()
Originals = dict()

basepath = '/home/julian/image_analysis'
path = 'data/whole_BP'

figure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))
filenames = []
for filename in os.listdir(os.path.join(basepath, path)):
    f = os.path.join(os.path.join(basepath, path), filename)
    # checking if it is a file
    if os.path.isfile(f):
        filenames.append(filename)

filenames = [
    ('E8/1_max_projection/E8_sample_1.tif', 'E8/1_max_projection/E8_sample_1__probability.tif'),
    ('E8/1_max_projection/E8_sample_2.tif', 'E8/1_max_projection/E8_sample_2__probability.tif'),

    ('E10/1_max_projection/E10_sample_2.tif', 'E10/1_max_projection/E10_sample_2__probability.tif'),
    ('E10_2025/3_cleaning/E10_2.tif', 'E10_2025/3_cleaning/E10_2__probability.tif'),
    ('E10_2025/3_cleaning/E10_3.tif', 'E10_2025/3_cleaning/E10_3__probability.tif'),
    # ('E10_2025/3_cleaning/E10_4.tif', 'E10_2025/3_cleaning/E10_4__probability.tif'),

    ('E12/1_max_projection/E12_sample_1.tif', 'E12/1_max_projection/E12_sample_1__probability.tif'),
    ('E12/1_max_projection/E12_sample_2.tif', 'E12/1_max_projection/E12_sample_2__probability.tif'),
    ('E12/1_max_projection/E12_sample_3.tif', 'E12/1_max_projection/E12_sample_3__probability.tif'),
    
    ('E14_2025/3_cleaning/E14_1.tif', 'E14_2025/3_cleaning/E14_1__probability.tif'),
    ('E14_2025/3_cleaning/E14_2.tif', 'E14_2025/3_cleaning/E14_2__probability.tif'),
    ('E14_2025/3_cleaning/E14_3.tif', 'E14_2025/3_cleaning/E14_3__probability.tif'),
]
# filenames = [f for f in filenames if f[-4:] == '.tif']
print(filenames)

for filename in filenames:
    if type(filename) is str:
        original = filename
    else:
        original = filename[0]
        if len(filename) > 1:
            filename = filename[1]
        else:
            filename = original
    print(filename, original)

    log.info("Loading image and segmentation to {} ...".format(filename))
    # continue

    Images[original] = load_image(
        base_dir = basepath,
        in_dir=path,
        filename=filename,
        dims=['y', 'x'],
        normalise=True,
        logger=log
    ).squeeze()
    Originals[original] = load_image(
        base_dir = basepath,
        in_dir=path,
        filename=original,
        dims=['y', 'x'],
        normalise=True,
        logger=log
    ).squeeze()

filenames = Images.keys()

In [ ]:
def subtract_background(image, radius=50, light_bg=False):
    from skimage.morphology import white_tophat, black_tophat, disk
    str_el = disk(radius) #you can also use 'ball' here to get a slightly smoother result at the cost of increased computing time
    if light_bg:
        return xr.DataArray(black_tophat(image, str_el), coords=image.coords)
    else:
        return xr.DataArray(white_tophat(image, str_el), coords=image.coords)

# Segment HCs
watershed on distance map

In [ ]:
def erode_label_interfaces(labels):
    labels[1:,:] = labels[1:,:].where(~np.multiply(labels.data[1:,:]!=labels.data[:-1,:], labels.data[:-1,:]>0), 0)
    # labels[:-1,:] = labels[:-1,:].where((__labels[:-1,:]==__labels[1:,:]) and (__labels[1:,:] > 0), 0)
    labels[:,1:] = labels[:,1:].where(~np.multiply(labels.data[:,1:]!=labels.data[:,:-1], labels.data[:,:-1] > 0), 0)
    # labels[:,:-1] = labels[:,:-1].where((__labels[:,:-1]==__labels[:,1:]) and (__labels[:,1:] > 0), 0)
    return labels

In [ ]:
def measure_cells(labels, *, voronoi=None, exclude_distance=2., minimum_area=5):
    res_x = labels.attrs.get('resolution_x', np.nan)
    res_y = labels.attrs.get('resolution_y', np.nan)
    res_z = labels.attrs.get('resolution_z', np.nan)

    objects = pd.DataFrame()
    objects['filename'] = None
    objects['stage'] = None
    objects['id'] = np.unique(labels)[1:]
    bincount = np.bincount(labels.data.flatten())
    objects['area_pixels'] = np.asarray([bincount[id] for id in objects['id']])
    objects['area_um2'] = objects['area_pixels'] / res_x / res_y
    objects['center_pixels'] = scipy.ndimage.measurements.center_of_mass(labels, labels=labels, index=objects['id'])
    objects['center_x_pixels'] = objects['center_pixels'].apply(lambda xy: int(np.round(xy[0])))
    objects['center_y_pixels'] = objects['center_pixels'].apply(lambda xy: int(np.round(xy[1])))
    objects['center_x_um'] = objects['center_x_pixels'] / res_x
    objects['center_y_um'] = objects['center_y_pixels'] / res_y

    # Excluding cells outside 2 sigma from local average area
    print(f"  Excluding cells outside {exclude_distance} sigma from local average area.")
    distance = scipy.spatial.distance_matrix(
        objects[['center_x_pixels', 'center_y_pixels']],
        objects[['center_x_pixels', 'center_y_pixels']]
    )
    distance_mask = distance < 500

    i = 0
    for index, row in objects.iterrows():
        objects.loc[index, 'local_area_pixel'] = objects.loc[distance_mask[i], 'area_pixels'].mean()
        objects.loc[index, 'local_area_pixel__std'] = objects.loc[distance_mask[i], 'area_pixels'].std()
        i += 1

    objects['exclude'] = (
          (objects['area_pixels'] < objects['local_area_pixel'] - exclude_distance * objects['local_area_pixel__std'])
        | (objects['area_pixels'] < minimum_area)
        # | (objects['area_pixels'] > objects['local_area_pixel'] + exclude_distance * objects['local_area_pixel__std'])
    )

    objects['filename'] = labels.attrs.get('filename', None)
    objects['stage'] = objects['filename'].apply(lambda s: s.split('_')[0])
    objects = objects.set_index(objects['id'])
    return objects

In [ ]:
Labels = dict()
Coords = dict()
Voronoi = dict()

for filename in filenames:
    print(filename)
    
    if 'z' in Images[filename].dims: 
        image = Images[filename].max(dim='z')
    else:
        image = Images[filename]
    img = image.data
    # img = skimage.filters.gaussian(img, sigma=0.5)
    N = 1
    while image.shape[0]/N > 2000:
        N += 1
    if N == 1:
        overlap = 0
    else:
        overlap = 200
    print(f"Splitting image ({image.shape[0]}x{image.shape[1]}) into {N} chunks with {overlap} pixels overlap")

    equalize= np.linspace(12, 10, N)
    peak_distance = np.linspace(8, 4, N)
    # peak_distance = np.linspace(8, 4, N) # E8 / E10 / E12
    if N == 1:
        equalize = [10]
        peak_distance = [10]

    __labels = xr.zeros_like(image, dtype=int)
    __voronoi = xr.zeros_like(image, dtype=int)
    markers = xr.zeros_like(image, dtype=int)
    distance = xr.zeros_like(image, dtype=int)

    # thresh = skimage.filters.threshold_otsu(image.data)
    # __bw = np.where(image > thresh/1.3, 1, 0)
    
    __bw = np.where(image > 0.7, 1, 0)
    # __bw = np.where(image > 0.1, 1, 0) # E10 / E12
    __bw = skimage.morphology.binary_dilation(__bw)

    for chunk in range(N):
    # for chunk in range(1,2):
        print(f"   Processing chunk {chunk+1}/{N} ...")
        Lx = image.shape[0]
        min_x = max(int(chunk/N*Lx-overlap/2), 0)
        max_x = min(int((chunk+1)/N*Lx+overlap/2), Lx)
        img = image.data[min_x:max_x,:]

        img = skimage.filters.gaussian(img, sigma=0.5)
        # img = skimage.morphology.white_tophat(img, skimage.morphology.disk(equalize[chunk]))
        # img = skimage.exposure.equalize_adapthist(img, kernel_size=equalize[chunk], clip_limit=0.01)


        bw = __bw[min_x:max_x].copy()
        bw[0,:] = 1
        bw[bw.shape[0]-1,:] = 1
        bw[:,0] = 1
        bw[:,bw.shape[1]-1] = 1
        bw = skimage.segmentation.flood_fill(bw, (0,0), 0)

        _distance = scipy.ndimage.distance_transform_edt(bw)

        coords = skimage.feature.peak_local_max(_distance, labels=bw, min_distance=int(peak_distance[chunk]))
        mask = np.zeros(img.shape, dtype=bool)
        mask[tuple(coords.T)] = True

        _markers, _ = scipy.ndimage.label(mask)

        _labels = np.zeros_like(img, dtype=int) + skimage.segmentation.watershed(-_distance, _markers, mask=bw)
        _voronoi = np.zeros_like(img, dtype=int) + skimage.segmentation.watershed(-_distance, _markers, mask=img)

        _labels = np.where(_labels > 0, _labels + __labels.max().data, 0)
        _voronoi = np.where(_voronoi > 0, _voronoi + __voronoi.max().data, 0)

        __labels[min_x:max_x,:] = np.where(__labels[min_x:max_x,:]==0, _labels, __labels[min_x:max_x,:])
        __voronoi[min_x:max_x,:] = np.where(__voronoi[min_x:max_x,:]==0, _voronoi, __voronoi[min_x:max_x,:])
        markers[min_x:max_x,:] = np.where(markers[min_x:max_x,:]==0, _markers, markers[min_x:max_x,:])
        distance[min_x:max_x,:] = np.where(distance[min_x:max_x,:]==0, _distance, distance[min_x:max_x,:])
    print(f"   Done.")

    labels = xr.zeros_like(__labels) + __labels
    voronoi = xr.zeros_like(__voronoi) + __voronoi
    ids = np.unique(__labels)[1:]
    # if np.unique(__voronoi)[1:].shape[0] != ids.shape[0]:
    #     raise

    # np.random.shuffle(ids)
    # for i, rid in enumerate(ids):
    #     labels = np.where(__labels==i+1, rid, labels)
    #     voronoi = np.where(__voronoi==i+1, rid, voronoi)

    labels = xr.zeros_like(image, dtype=int) + erode_label_interfaces(labels)
    voronoi = xr.zeros_like(image, dtype=int) + voronoi
    markers = xr.zeros_like(image, dtype=bool) + markers > 0
    distance = xr.zeros_like(image) + distance
    labels.attrs = image.attrs
    voronoi.attrs = image.attrs
    markers.attrs = image.attrs
    distance.attrs = image.attrs

    # Labels[filename] = labels
    # Coords[filename] = coords
    # Voronoi[filename] = voronoi

    objects = measure_cells(labels, exclude_distance=5, minimum_area=50)
    cnt_exclude = objects['exclude'].count()
    print(f'Found {cnt_exclude} cells in {filename}.')
    
    cnt_exclude = objects['exclude'].sum()
    print(f'Excluding {cnt_exclude} cells based on local 2 std area criterion.')

    for id in objects.loc[objects['exclude'], 'id']:
        labels = labels.where(labels!=id, 0)

    # Labels[filename] = labels
    # Coords[filename] = coords
    # Voronoi[filename] = voronoi

    print(f'Identified {np.unique(labels).shape[0]-1} cells in {filename}.')

    if filename in Originals.keys():
        original = Originals[filename]
    else:
        original = Images[filename]
    tmp = xr.concat([original, labels.where(labels==0, labels % 20 + 1)], dim='c').assign_coords(c=[0,1])
    tmp.attrs = image.attrs

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    if not 'stage' in objects.columns:
        objects['stage'] = filename.split('/')[0].split('_')[0]
        print(objects['stage'])

    print(f"Saving result ...")
    objects.to_csv(os.path.join(out_dir, f"{filename_raw}.csv"))
    save_image(
        tmp,
        normalise=True,
        base_dir=None,
        out_dir=out_dir,
        filename=filename_raw+'__labels.tif'
    )

    continue

    # if E14
    image_prox = image.data[:int(Lx/2),:]
    image_prox = skimage.filters.gaussian(image_prox, sigma=0.5)
    image_prox = skimage.morphology.white_tophat(image_prox, skimage.morphology.disk(15))
    image_prox = skimage.exposure.equalize_adapthist(image_prox, kernel_size=15, clip_limit=0.01)

    image_dist = image.data[int(Lx/2):,:]
    image_dist = skimage.filters.gaussian(image_dist, sigma=0.5)
    image_dist = skimage.morphology.white_tophat(image_dist, skimage.morphology.disk(10))
    image_dist = skimage.exposure.equalize_adapthist(image_dist, kernel_size=10, clip_limit=0.01)

    img = np.zeros_like(image.data)
    img[:int(Lx/2),:] = image_prox
    img[int(Lx/2):,:] = image_dist

    # Else
    #     img = image.data
    #     img = skimage.filters.gaussian(img, sigma=0.5)
    #     img = skimage.morphology.white_tophat(img, skimage.morphology.disk(10))
    #     img = skimage.exposure.equalize_adapthist(img, kernel_size=10, clip_limit=0.01)



    fig, axes = plt.subplots(1, 2, figsize=[20, 20])

    axes[0].imshow(image.T, interpolation='none', cmap='gray')

    ax = axes[1]
    ax.set_aspect(1)
    ax.imshow(img.T, cmap='gray')


    thresh = skimage.filters.threshold_otsu(img)

    bw = np.where(img > thresh/1.3, 1, 0)
    bw = skimage.morphology.binary_dilation(bw)

    distance = scipy.ndimage.distance_transform_edt(bw)
    
    
    # if E14
    coords = skimage.feature.peak_local_max(distance[:int(Lx/4),:], labels=bw[:int(Lx/4),:], min_distance=16)
    mask_prox = np.zeros(distance[:int(Lx/4),:].shape, dtype=bool)
    mask_prox[tuple(coords.T)] = True

    coords = skimage.feature.peak_local_max(distance[int(Lx/4):int(Lx*3/4),:], labels=bw[int(Lx/4):int(Lx*3/4),:], min_distance=12)
    mask_mid = np.zeros(distance[int(Lx/4):int(Lx*3/4),:].shape, dtype=bool)
    mask_mid[tuple(coords.T)] = True

    coords = skimage.feature.peak_local_max(distance[int(Lx*3/4):,:], labels=bw[int(Lx*3/4):,:], min_distance=8)
    mask_dist = np.zeros(distance[int(Lx*3/4):,:].shape, dtype=bool)
    mask_dist[tuple(coords.T)] = True

    mask = np.zeros_like(distance, dtype=bool)
    mask[:int(Lx/4),:] = mask_prox
    mask[int(Lx/4):int(Lx*3/4),:] = mask_mid
    mask[int(Lx*3/4):,:] = mask_dist

    # else
    # coords = skimage.feature.peak_local_max(distance, labels=bw, min_distance=10)
    # mask = np.zeros(distance.shape, dtype=bool)
    # mask[tuple(coords.T)] = True


    markers, _ = scipy.ndimage.label(mask)


    __labels = xr.zeros_like(image, dtype=int)
    __labels[:int(Lx/3),:] += skimage.segmentation.watershed(-distance[:int(Lx/3),:], markers[:int(Lx/3),:], mask=bw[:int(Lx/3),:])
    __voronoi = xr.zeros_like(image, dtype=int) # + skimage.segmentation.watershed(-distance, markers, mask=image)

    labels = xr.zeros_like(__labels) + __labels
    voronoi = xr.zeros_like(__voronoi) + __voronoi
    ids = np.unique(__labels)[1:]
    # if np.unique(__voronoi)[1:].shape[0] != ids.shape[0]:
    #     raise

    # np.random.shuffle(ids)
    # for i, rid in enumerate(ids):
    #     labels = np.where(__labels==i+1, rid, labels)
    #     voronoi = np.where(__voronoi==i+1, rid, voronoi)

    labels = xr.zeros_like(image, dtype=int) + labels
    voronoi = xr.zeros_like(image, dtype=int) + voronoi
    markers = xr.zeros_like(image, dtype=bool) + markers > 0
    distance = xr.zeros_like(image) + distance
    labels.attrs = image.attrs
    voronoi.attrs = image.attrs
    markers.attrs = image.attrs
    distance.attrs = image.attrs

    Labels[filename] = labels
    Coords[filename] = coords
    Voronoi[filename] = voronoi

    print(f'Found {np.unique(labels).shape[0]-1} cells in {filename}.')

    tmp = xr.concat([image, markers, labels, labels%15, distance], dim='c').assign_coords(c=[0,1,2,3,4])
    tmp.attrs = image.attrs

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    save_image(
        tmp,
        normalise=True,
        base_dir=None,
        out_dir=out_dir,
        filename=filename_raw+'__labels.tif'
    )

image=None
del image
img=None
del img
thresh=None
del thresh
bw=None
del bw
distance=None
del distance
coords=None
del coords
mask=None
del mask
markers=None
del markers
labels=None
del labels
__labels=None
del __labels
voronoi=None
del voronoi

In [ ]:
print(np.bincount(labels.data.flatten()).shape)
print(np.unique(labels.data.flatten()).shape)
# print(measure_cells(labels))

In [ ]:
raise RuntimeError("Please apply manual corrections and create a '<filename>__labels__manual_corrections.tif' file before continuing!")

In [ ]:
Manuals = dict()
for filename in Images.keys():
    image = Images[filename].copy()
    if filename in Originals.keys():
        original = Originals[filename].copy()
    else:
        original = Images[filename].copy()

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)

    try:
        tmp = load_image(
            base_dir = None,
            in_dir=out_dir,
            filename=f'{filename_raw}__labels__manual_corrections.tif',
            dims=['c', 'y', 'x'],
            normalise=True,
            logger=log
        ).squeeze()
        mask = tmp.isel(c=1) > 0
        # distance = scipy.ndimage.distance_transform_edt(mask)
        tmp = None
        del tmp
    except RuntimeError as err:
        print(f"ERROR loading manual corrections for {filename}:")
        print(err)
        continue
    
    # coords = skimage.feature.peak_local_max(distance, labels=mask, min_distance=3)
    # seeds = np.zeros(image.shape, dtype=bool)
    # seeds[tuple(coords.T)] = True

    # markers, _ = scipy.ndimage.label(seeds)

    # _labels = np.zeros_like(image, dtype=int) + skimage.segmentation.watershed(-distance, markers, mask=mask)

    # labels = xr.zeros_like(image, dtype=int) + _labels
    labels = xr.zeros_like(image, dtype=int) + scipy.ndimage.label(mask.data)[0]
    voronoi = xr.zeros_like(image, dtype=int) # + _labels
    # markers = (xr.zeros_like(image, dtype=int) + markers).where(labels > 0, 0)
    # distance = xr.zeros_like(image, dtype=int) + distance
    labels.attrs = original.attrs
    # voronoi.attrs = original.attrs
    # markers.attrs = original.attrs
    # distance.attrs = original.attrs

    objects = measure_cells(labels, exclude_distance=np.inf, minimum_area=1)

    print(f'Found {np.unique(labels).shape[0]-1} cells in {filename}.')
    cnt_exclude = objects['exclude'].sum()
    print(f'Excluding {cnt_exclude} cells based on local 2 std area criterion.')

    # for id in objects.loc[objects['exclude'], 'id']:
    #     labels = labels.where(labels!=id, 0)

    # Labels[filename] = labels
    # Coords[filename] = coords
    # Voronoi[filename] = voronoi

    print(f'Identified {np.unique(labels).shape[0]-1} cells in {filename}.')

    tmp = xr.concat([original, labels.where(labels==0, labels % 20 + 6)], dim='c', coords='minimal').assign_coords(c=[0,1])

    print(f"Saving result ...")
    objects.to_csv(os.path.join(out_dir, f"{filename_raw}.csv"))
    save_image(
        tmp,
        normalise=True,
        base_dir=None,
        out_dir=out_dir,
        filename=filename_raw+'__labels.tif'
    )

In [ ]:
raise RuntimeError("Please finalise manual corrections and create a '<filename>__labels__final.tif' file before continuing!")

In [ ]:
print(objects['exclude'].sum())

# Calculate on identified cells

In [ ]:
def voronoi_neighbors(voronoi, *, objects):
    nbs = []
    Lx, Ly = voronoi.shape
    Lx -= 1
    Ly -= 1
    for i, cell in objects.iterrows():
        id = cell['id']
        x = cell['center_x_pixels']
        y = cell['center_y_pixels']

        voronoi_nbs = voronoi[max(0, x-50):min(Lx, x+50), max(0, y-50):min(Ly, y+50)].where(
            # get all the pixels adjacent to voronoi cell
            skimage.morphology.binary_dilation(voronoi.data[max(0, x-50):min(Lx, x+50), max(0, y-50):min(Ly, y+50)] == id, np.ones((3,3)))  ^ (voronoi.data[max(0, x-50):min(Lx, x+50), max(0, y-50):min(Ly, y+50)] == id), 
            0
        )
        
        if voronoi_nbs.sum() == 0:
            # print(f"No voronoi nbs found for cell {cell['id']}")
            nbs.append('')
            continue

        _nbs = ''
        cnt = 0
        for n in np.unique(voronoi_nbs)[1:]:
            cnt += 1
            _nbs += str(n) + ','
        if cnt >= 3:
            nbs.append(_nbs[:-1])
        else:
            nbs.append('')

    return np.asarray(nbs)

In [ ]:
import numbers

def r_ellipse(theta, a, b, theta0):
    return a * b / (  (b * np.cos(theta - theta0))**2
                    + (a * np.sin(theta - theta0))**2)**0.5

def p_atic_order_parameter(displ_x, displ_y, *, p, corrected=False):
    def __rescale_dx_dy(displ_x, displ_y, *, ratio_WH, theta):
        """Rescale to circle for correction"""
        dx_ = (displ_x * np.cos(-theta) - displ_y * np.sin(-theta)) / ratio_WH
        dy_ =  displ_x * np.sin(-theta) + displ_y * np.cos(-theta)

        dx = dx_ * np.cos(theta) - dy_ * np.sin(theta)
        dy = dx_ * np.sin(theta) + dy_ * np.cos(theta)

        return dx, dy
    

    if displ_x.size < 3:
        if isinstance(p, numbers.Number):
            return 0.
        else:
            return np.zeros_like(p)
    
    if not corrected:
        angles = np.arctan2(displ_y, displ_x)
    else:
        p0 = 1.01, 0.99, 0.01
        try: 
            popt, _ = curve_fit(r_ellipse,
                                np.arctan2(displ_y, displ_x), (displ_x**2 + displ_y**2)**0.5,
                                p0, sigma = (displ_x**2 + displ_y**2)**0.5)
            A, B, Alpha = popt
            dx, dy = __rescale_dx_dy(displ_x, displ_y, ratio_WH=A/B, theta=Alpha)
            angles = np.arctan2(dy, dx)
        except RuntimeError as err:
            print(err)
            angles = np.arctan2(displ_y, displ_x)

    if isinstance(p, numbers.Number):
        return np.abs((np.exp(p * 1j * angles)).sum()) / angles.shape[0]
    else:
        return np.asarray(
            [
                np.abs((np.exp(_p * 1j * angles)).sum()) / angles.shape[0]
                for _p in p
            ]
        )

In [ ]:
def p_atic_order(cell, *, objects, ps):
    if type(cell['neighbors_voronoi']) is str:
        if len(cell['neighbors_voronoi']) == 0:
            nbs = []
        else:
            nbs = cell['neighbors_voronoi'].split(',')
    else:
        nbs = cell['neighbors_voronoi']
    
    nbs = np.asarray(nbs, dtype=int)
    
    nbs_x = np.asarray([objects.loc[n, 'center_x_pixels'] for n in nbs])
    nbs_y = np.asarray([objects.loc[n, 'center_y_pixels'] for n in nbs])

    displ_x = nbs_x - cell['center_x_pixels']
    displ_y = nbs_y - cell['center_y_pixels']


    return nbs, p_atic_order_parameter(displ_x, displ_y, p=ps, corrected=False), p_atic_order_parameter(displ_x, displ_y, p=ps, corrected=True)

def apply_p_atic_order(objects, *, ps):
    tmp_result = objects.apply(p_atic_order, axis=1, objects=objects, ps=ps)

    result = pd.DataFrame()    
    for i, p in enumerate(ps):
        if i == 0:
            result['next_HC_neighbors'] = tmp_result.apply(lambda r: np.asarray(r[0]).astype(int))
            result['num_next_HC_neighbors'] = tmp_result.apply(lambda r: len(r[0]))

        result[f'{p}-atic_order'] = tmp_result.apply(lambda r: r[1][i])
        result[f'{p}-atic_order_corrected'] = tmp_result.apply(lambda r: r[2][i])
        if p == 6:
            result['hexatic_order'] = tmp_result.apply(lambda r: r[1][i])
            result['hexatic_order_corrected'] = tmp_result.apply(lambda r: r[2][i])
    return result

In [ ]:
def circular_mask(radius):
    x = np.linspace(-radius, radius, 2*radius+1)  # Adjust bounds to include the circle
    y = np.linspace(-radius, radius, 2*radius+1)
    X, Y = np.meshgrid(x, y)

    # Calculate the distance from the center (0, 0)
    distance = np.sqrt(X**2 + Y**2)

    # Create the mask: 1 inside the circle, 0 outside
    return (distance <= radius).astype(bool)


In [ ]:
Labels = dict()
Voronoi = dict()
Objects = dict()
for filename in Images.keys():
    print(filename)

    image = Images[filename].copy()
    if filename in Originals.keys():
        original = Originals[filename].copy()
    else:
        original = Images[filename].copy()

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)

    try:
        tmp = load_image(
            base_dir = None,
            in_dir=out_dir,
            filename=f'{filename_raw}__labels__final.tif',
            dims=['c', 'y', 'x'],
            normalise=True,
            logger=log
        ).squeeze()
        mask = tmp.isel(c=1) > 0
        tmp = None
        del tmp
    except RuntimeError as err:
        print(f"ERROR loading manual corrections for {filename}:")
        print(err)
        continue

    labels = xr.zeros_like(image, dtype=int) + scipy.ndimage.label(mask.data)[0]
    labels.attrs = original.attrs

    Labels[filename] = labels

    objects = measure_cells(labels, exclude_distance=np.inf, minimum_area=1)
    objects['exclude'] = False
    print(f'Identified {np.unique(labels).shape[0]-1} cells in {filename}.')

    # voronoi tesselation for hexatic order
    coords = np.asarray(np.round([[xy[0], xy[1]] for xy in objects['center_pixels']]), dtype=int)
    seeds = np.zeros(image.shape, dtype=bool)
    seeds[tuple(coords.T)] = True
    markers = labels.where(seeds, 0).data

    distance = scipy.ndimage.distance_transform_edt(mask)
    voronoi = xr.zeros_like(image, dtype=int) + skimage.segmentation.watershed(-distance, markers, mask=original)
    voronoi.attrs = original.attrs
    Voronoi[filename] = voronoi

    objects['neighbors_voronoi'] = voronoi_neighbors(voronoi, objects=objects)
    objects = pd.concat([objects, apply_p_atic_order(objects=objects, ps=range(3, 10))], axis=1)

    print(f"Saving result ...")
    objects.to_csv(os.path.join(out_dir, f"{filename_raw}__final.csv"))
    Objects[filename] = objects

    psi_6 = objects['hexatic_order'].mean()
    print(f'Mean hexatic order: {psi_6}')

In [ ]:
# Objects = pd.DataFrame()
# Objects['filename'] = ''

# def hexatic_order(cell, *, objects):
#     nbs = cell['neighbors_voronoi']

#     if len([n for n in nbs if not objects.loc[objects['id']==n].empty]) < 3:
#         return 0.

#     nbs_x = np.asarray([objects.loc[n, 'barycenter_x_pixels'] for n in nbs if not objects.loc[objects['id']==n].empty])
#     nbs_y = np.asarray([objects.loc[n, 'barycenter_y_pixels'] for n in nbs if not objects.loc[objects['id']==n].empty])

#     return np.abs(np.exp(6j * np.arctan2(nbs_y - cell['barycenter_y_pixels'], nbs_x - cell['barycenter_x_pixels'])).sum()) / nbs_x.size

# # TODO replace with the respective function in utils.py
# def elongation(coordinates, *, barycenter):
#     phi = np.arctan2(coordinates[:,1]-barycenter[1],
#                      coordinates[:,0]-barycenter[0])
    
#     q1 = (np.cos(2*phi)).sum() / coordinates.shape[0]
#     q2 = (np.sin(2*phi)).sum() / coordinates.shape[0]

#     q = (q1**2 + q2**2)**0.5
#     orientation = np.arctan2(q2, q1) / 2
    
#     return q, orientation, q1, q2


# # TODO replace with the respective function in utils.py
# def HCA_polarity(coordinates, *, intensity, barycenter):
#     phi = np.arctan2(coordinates[:,1]-barycenter[1],
#                      coordinates[:,0]-barycenter[0])
    
#     px = (intensity * np.cos(phi)).sum() / intensity.sum()
#     py = (intensity * np.sin(phi)).sum() / intensity.sum()

#     p = (px**2 + py**2)**0.5
#     orientation = np.arctan2(py, px)
    
#     return p, orientation, px, py

# # def local_tangent_to_axis(coordinates, *, )

# # analyse the objects
# for filename in filenames:
#     print(f'processing {filename} ...')
#     _labels = Labels[filename]
#     if 'z' in Images[filename].dims:
#         image = Images[filename].max(dim='z')
#     else:
#         image = Images[filename]
#     image.attrs = Images[filename].attrs
#     _voronoi = Voronoi[filename]
#     objects = pd.DataFrame()
#     objects['filename'] = ''
#     objects['id'] = 0

#     size = 200
#     overlap = 50

#     start_time = time.time()
#     report_time = start_time
#     for x in range(0, _labels.shape[0], size):
#         if time.time() - report_time > 10:
#             report_time = time.time()
#             print(f'  progress: {x/_labels.shape[0]} in {time.time() - start_time}s')
#         for y in range(0, _labels.shape[1], size):
#             labels = _labels.isel(
#                 x=slice(np.max([x-overlap, 0]), x+size+overlap),
#                 y=slice(np.max([y-overlap, 0]), y+size+overlap),
#             )
#             voronoi = _voronoi.isel(
#                 x=slice(np.max([x-overlap, 0]), x+size+overlap),
#                 y=slice(np.max([y-overlap, 0]), y+size+overlap),
#             )

#             for id in np.unique(labels):
#                 if id == 0 or not objects.loc[objects['id']==id].empty:
#                     continue

#                 cell = labels.where(labels == id)

#                 # exclude cells on boundary of image
#                 if (
#                     np.count_nonzero(~np.isnan(cell[0, :])) + 
#                     np.count_nonzero(~np.isnan(cell[1, :])) + 
#                     np.count_nonzero(~np.isnan(cell[-1, :])) +
#                     np.count_nonzero(~np.isnan(cell[-2, :])) +
#                     np.count_nonzero(~np.isnan(cell[:, 0])) + 
#                     np.count_nonzero(~np.isnan(cell[:, 1])) + 
#                     np.count_nonzero(~np.isnan(cell[:, -1])) +
#                     np.count_nonzero(~np.isnan(cell[:, -2]))
#                 ):
#                     continue

#                 objects.loc[id, 'id'] = id

#                 # the coordinate axes
#                 coordinates = np.argwhere((labels == id).data)
#                 coordinates = np.asarray(coordinates, dtype=float)
#                 coordinates = coordinates + np.array([np.max([x-overlap, 0]), 
#                                                       np.max([y-overlap, 0])])
                              
                
#                 # calculate in pixel dimensions
#                 barycenter = coordinates.mean(0)
#                 max = coordinates.max(0)
#                 min = coordinates.min(0)

#                 mask = np.where(labels == id, 1, 0)
#                 area = mask.sum()

#                 objects.loc[id, 'area_pixels'] = area
#                 for i, axis in enumerate(['x', 'y']):
#                     objects.loc[id, 'barycenter_{}_pixels'.format(axis)] = barycenter[i]
#                 for i, axis in enumerate(['x', 'y']):
#                     objects.loc[id, 'min_{}_pixels'.format(axis)] = min[i]
#                     objects.loc[id, 'max_{}_pixels'.format(axis)] = max[i]

#                 Q = elongation(coordinates, barycenter=barycenter)
#                 objects.loc[id, 'elongation'] = Q[0]
#                 objects.loc[id, 'elongation_orientation'] = Q[1]
#                 objects.loc[id, 'elongation_q1'] = Q[2]
#                 objects.loc[id, 'elongation_q2'] = Q[3]

#                 # A list of intensities sorted like coordinates
#                 intensity = image.isel(
#                         x=slice(np.max([x-overlap, 0]), x+size+overlap),
#                         y=slice(np.max([y-overlap, 0]), y+size+overlap),
#                     ).data[labels==id]  
#                 HCA_p = HCA_polarity(coordinates, intensity=intensity, barycenter=barycenter)

#                 objects.loc[id, 'HCA_polarity'] = HCA_p[0]
#                 objects.loc[id, 'HCA_polarity_orientation'] = HCA_p[1]
#                 objects.loc[id, 'HCA_polarity_px'] = HCA_p[2]
#                 objects.loc[id, 'HCA_polarity_py'] = HCA_p[3]

#                 voronoi_nbs = voronoi.where(
#                     # get all the pixels adjacent to voronoi cell
#                     skimage.morphology.binary_dilation(voronoi.data == id),
#                     0
#                 )

#                 _nbs = ''
#                 cnt = 0
#                 for n in np.unique(voronoi_nbs):
#                     if n != 0 and n != id:
#                         cnt += 1
#                         _nbs += str(n) + ','
#                 if cnt >= 3:
#                     objects.loc[id, 'neighbors_voronoi'] = _nbs[:-1]
#                 else:
#                     objects.loc[id, 'neighbors_voronoi'] = '0,0,0'
#     print(f'Measured HCs in {time.time() - start_time}s. Postprocessing ... ')

#     # use image resolution
#     res_x = image.attrs.get('resolution_x', np.nan)
#     res_y = image.attrs.get('resolution_y', np.nan)
#     res_z = image.attrs.get('resolution_z', np.nan)

#     aspect = res_x / res_y
#     print(f"{image.attrs['resolution_scale']}: {res_x} x {res_y} ({aspect})")

#     objects['area'] = objects['area_pixels'] / res_x / res_y

#     objects['barycenter_x'] = objects['barycenter_x_pixels'] / res_x
#     objects['barycenter_y'] = objects['barycenter_y_pixels'] / res_y

#     objects['min_x'] = objects['min_x_pixels'] / res_x
#     objects['max_x'] = objects['max_x_pixels'] / res_x
#     objects['min_y'] = objects['min_y_pixels'] / res_y
#     objects['max_y'] = objects['max_y_pixels'] / res_y

#     objects['neighbors_voronoi'] = objects['neighbors_voronoi'].apply(lambda s: np.asarray(s.split(','), dtype=int))
#     objects['num_neighbors_voronoi'] = objects['neighbors_voronoi'].apply(lambda nbs: nbs.size)

#     print("  Calculating hexatic order ...")
#     objects['hexatic_order'] = objects[['neighbors_voronoi', 'barycenter_x_pixels', 'barycenter_y_pixels']].apply(hexatic_order, axis=1, objects=objects)
#     print("  Done.")

#     objects['filename'] = filename

#     # Excluding cells outside 2 sigma from local average area
#     print("  Excluding cells outside 2 sigma from local average area.")
#     distance = scipy.spatial.distance_matrix(
#         objects[['barycenter_x_pixels', 'barycenter_y_pixels']],
#         objects[['barycenter_x_pixels', 'barycenter_y_pixels']]
#     )
#     distance_mask = distance < 500

#     i = 0
#     for index, row in objects.iterrows():
#         objects.loc[index, 'local_area_pixel'] = objects.loc[distance_mask[i], 'area_pixels'].mean()
#         objects.loc[index, 'local_area_pixel__std'] = objects.loc[distance_mask[i], 'area_pixels'].std()
#         i += 1

#     objects['exclude'] = (
#           (objects['area_pixels'] < objects['local_area_pixel'] - 2 * objects['local_area_pixel__std'])
#         | (objects['area_pixels'] > objects['local_area_pixel'] + 2 * objects['local_area_pixel__std'])
#     )
    
#     filename_raw = filename.split('/')[-1].split('.')[0]
#     out_dir = os.path.join(basepath, path)
#     for p in  filename.split('/')[:-1]:
#         out_dir = os.path.join(out_dir, p)
#     out_dir = os.path.join(out_dir, filename_raw)

#     objects.to_csv(os.path.join(out_dir, f"{filename_raw}.csv"))

#     Objects = pd.concat([Objects, objects])
#     print(f'Finished file {filename} in {time.time() - start_time}s.')

# print(Objects[['filename', 'area', 'num_neighbors_voronoi', 'hexatic_order']].groupby(by='filename').mean())

# _labels=None
# del _labels
# image=None
# del image
# _voronoi=None
# del _voronoi
# objects=None
# del objects
# labels=None
# del labels
# voronoi=None
# del voronoi
# cell=None
# del cell
# coordinates=None
# del coordinates
# intensity=None
# del intensity
# mask=None
# del mask
# voronoi_nbs=None
# del voronoi_nbs
# distance=None
# del distance
# distance_mask=None
# del distance_mask

# Measure curvature in images

In [ ]:
def PD_position(data):
    result = data['curvature_tangent_orientation']
    result -= result.min()
    result /= result.max()
    return result

def PD_binned_position(tangent, bins=[0., 0.125, 0.375, 0.625, 0.875, 1.0], digits=3):
    # arguments:
    #   - bins: the right edges of the bins
    binned = np.zeros_like(tangent)
    bins[-1] += 1.e-8
    for i, bin in enumerate(bins[:-1]):
        binned += (tangent >= bin) * (tangent < bins[i+1]) * (bins[i+1] + bins[i]) / 2.
    binned = np.round(binned * 10**digits) / 10**digits
    return binned

In [ ]:
def calc_R(xc, yc):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def f_2b(c):
    """ calculate the algebraic distance between the 2D points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(*c)
    return Ri - Ri.mean()

def Df_2b(c):
    """ Jacobian of f_2b
    The axis corresponding to derivatives must be coherent with the col_deriv option of leastsq"""
    xc, yc     = c
    df2b_dc    = np.empty((len(c), x.size))

    Ri = calc_R(xc, yc)
    df2b_dc[0] = (xc - x)/Ri                   # dR/dxc
    df2b_dc[1] = (yc - y)/Ri                   # dR/dyc
    df2b_dc    = df2b_dc - df2b_dc.mean(axis=1)[:, np.newaxis]

    return df2b_dc

figsize=8
for filename in filenames:
# for filename in ['E14_2025/3_cleaning/E14_1.tif']:
    print(filename)

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)

    if os.path.isfile(os.path.join(out_dir, f"{filename_raw}__final.csv")):
        objects = pd.read_csv(os.path.join(out_dir, f"{filename_raw}__final.csv"))
        if 'Unnamed: 0' in objects.columns:
            objects = objects.drop(columns='Unnamed: 0')
        objects.set_index('id')
    else:
        raise
        objects = Objects[filename]

    labels = Labels[filename]    
        
    image = Images[filename].copy()
    if filename in Originals.keys():
        original = Originals[filename].copy()
    else:
        original = Images[filename].copy()
    img = original.copy()

    x = np.asarray(objects['center_x_um'])
    y = np.asarray(objects['center_y_um'])
    center_estimate = x.mean(), y.mean()
    center_2b, ier = scipy.optimize.leastsq(f_2b, center_estimate, Dfun=Df_2b, col_deriv=True)

    xc_2b, yc_2b = center_2b
    Ri_2b        = calc_R(*center_2b)
    R_2b         = Ri_2b.mean()
    residu_2b    = sum((Ri_2b - R_2b)**2)

    print(f'Radius: {R_2b} $\mu m$')
    print(f'curvature: {1/R_2b} $\mu m^-1$')
    print(f'curvature: {1/(R_2b/2.8)} $A^-0.5$')

    tissue = pd.Series()

    tissue['filename'] = filename
    tissue['stage'] = filename.split('/')[-1].split('_')[0]
    tissue['curvature_center_x_um'] = xc_2b
    tissue['curvature_center_y_um'] = yc_2b
    tissue['curvature_radius_um'] = R_2b


    x = np.asarray(objects['center_x_pixels'])
    y = np.asarray(objects['center_y_pixels'])
    center_estimate = x.mean(), y.mean()
    center_2b, ier = scipy.optimize.leastsq(f_2b, center_estimate, Dfun=Df_2b, col_deriv=True)

    xc_2b, yc_2b = center_2b
    Ri_2b        = calc_R(*center_2b)
    R_2b         = Ri_2b.mean()
    residu_2b    = sum((Ri_2b - R_2b)**2)

    tissue['curvature_center_x_pixels'] = xc_2b
    tissue['curvature_center_y_pixels'] = yc_2b
    tissue['curvature_radius_pixels'] = R_2b


    fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])

    ax.set_aspect(1)
    if img is not None:
        ax.imshow(img.T, cmap='gray')
    else:
        ax.scatter(x, y, s=1., linewidth=0.)

    ax.plot(
        R_2b * np.cos(np.linspace(0, 2*math.pi, 101)) + xc_2b, 
        R_2b * np.sin(np.linspace(0, 2*math.pi, 101)) + yc_2b,
        label="curvature: %.2e $\mu m^{-1}$ / %.2e $A^{-0.5}$"%((1/tissue['curvature_radius_um']), 1/(tissue['curvature_radius_um']/objects['area_um2'].mean()**0.5))
    )
    ax.legend()
    if img is not None:
        ax.set_xlim(0, img.shape[0])
        ax.set_ylim(img.shape[1], 0)
    else:
        ax.set_xlim(x.min(), x.max())
        ax.set_ylim(y.max(), y.min())

    if img is not None:
        if not os.path.isdir(os.path.join(out_dir, "plots")):
            os.makedirs(os.path.join(out_dir, "plots"))
        fig.savefig(os.path.join(out_dir, "plots", f"{filename_raw}__curvature.svg"),format='svg')

    objects['curvature_tangent_orientation'] = np.arctan2(objects['center_y_um'] - tissue['curvature_center_y_um'], objects['center_x_um'] - tissue['curvature_center_x_um']) + math.pi/2
    objects['position_PD'] = PD_position(objects)
    objects['position_PD__binned'] = objects['position_PD'].apply(PD_binned_position, bins=[0., 0.125, 0.375, 0.625, 0.875, 1.0])
    objects['position_PD__binned_20'] = objects['position_PD'].apply(PD_binned_position, bins=[0., 0.2, 0.4, 0.6, 0.8, 1.0])
    objects['position_PD__binned_12.5'] = objects['position_PD'].apply(PD_binned_position, bins=[0., 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0])

    objects.to_csv(os.path.join(out_dir, f"{filename_raw}__final.csv"))
    tissue.to_csv(os.path.join(out_dir, f"{filename_raw}__tissue_properties.csv"))

    Objects[filename] = objects



objects=None
del objects
img=None
del img
x=None
del x
y=None
del y

Ri_2b = None
del Ri_2b

## Plot Cell properties

In [ ]:
print(Objects.keys())
print(Labels.keys())
print(Voronoi.keys())

In [ ]:
print(np.iinfo(np.short).max)

In [ ]:
def prepare_save_tif(image, *, normalise):
    image = image.copy()
    if normalise:
        image -= image.min()
        image /= image.max()
        image *= np.iinfo(np.short).max
        image = image.astype(dtype='uint16')
        __image = None
        del __image
    else:
        if (image.max() <= 1.):
            raise RuntimeError("Cannot convert image with values <= 1 to uint16")
        image = image.astype(dtype='uint16')
    return image

In [ ]:
print(original.attrs)

In [ ]:
for filename in filenames:
# for filename in ['E14_2025/3_cleaning/E14_1.tif']:
    print(filename)

    filename_raw = filename.split('/')[-1].split('.')[0]
    out_dir = os.path.join(basepath, path)
    for p in  filename.split('/')[:-1]:
        out_dir = os.path.join(out_dir, p)
    out_dir = os.path.join(out_dir, filename_raw)

    if filename in Objects.keys():
        objects = Objects[filename]
        
    elif os.path.isfile(os.path.join(out_dir, f"{filename_raw}__final.csv")):
        objects = pd.read_csv(os.path.join(out_dir, f"{filename_raw}__final.csv"))
        if 'Unnamed: 0' in objects.columns:
            objects = objects.drop(columns='Unnamed: 0')
        objects['center_pixels'] = objects['center_pixels'].apply(lambda s: np.asarray(s[1:-1].split(', '), dtype=float))
    else:
        raise
    
    labels = Labels[filename]    
    voronoi = Voronoi[filename]

    image = Images[filename].copy()
    if filename in Originals.keys():
        original = Originals[filename].copy()
    else:
        original = Images[filename].copy()

    res_x = original.attrs['resolution_x']
    res_y = original.attrs['resolution_y']
    aspect = res_x / res_y
    extent=[0, original.shape[0]/res_x, 0, original.shape[1]/res_y]
    print(f"{original.attrs['resolution_scale']}: {res_x} x {res_y} ({aspect})")
    if not np.isfinite(aspect):
        raise RuntimeError(f"No resolution available for image {filename}!")
    
    area_map = xr.zeros_like(labels, dtype=float)
    hexatic_map = xr.zeros_like(labels, dtype=float)
    hexatic_map_c = xr.zeros_like(labels, dtype=float)
    neighbour_map = xr.zeros_like(labels, dtype=float)
    density_map = xr.zeros_like(labels, dtype=float)

    # create a mask of the tissue
    coords = np.asarray(np.round([[xy[0], xy[1]] for xy in objects['center_pixels']]), dtype=int)
    seeds = np.zeros(image.shape, dtype=bool)
    seeds[tuple(coords.T)] = True
    tissue_map = xr.zeros_like(labels, dtype=float) + scipy.ndimage.binary_dilation(seeds, circular_mask(50))

    Lx, Ly = original.shape
    for id, cell in objects.iterrows():
        if id % 500 == 0:
            print(f"Progress  : {id / objects['id'].max()} ...")
        x = cell['center_x_pixels']
        y = cell['center_y_pixels']
        x0, x1 = max(0, x - 50), min(Lx, x + 50 + 1)
        y0, y1 = max(0, y - 50), min(Ly, y + 50 + 1)
        area_map[x0:x1, y0:y1] = area_map[x0:x1, y0:y1].where(
            labels[x0:x1, y0:y1] != id, cell['area_pixels'])
        hexatic_map[x0:x1, y0:y1] = hexatic_map[x0:x1, y0:y1].where(
            labels[x0:x1, y0:y1] != id, cell['hexatic_order'])
        hexatic_map_c[x0:x1, y0:y1] = hexatic_map_c[x0:x1, y0:y1].where(
            labels[x0:x1, y0:y1] != id, cell['hexatic_order_corrected'])
        # neighbour_map[x0:x1, y0:y1] = neighbour_map[x0:x1, y0:y1].where(
        #     labels[x0:x1, y0:y1] != id, cell['num_next_HC_neighbors']) 
        neighbour_map[x0:x1, y0:y1] = neighbour_map[x0:x1, y0:y1].where(
            labels[x0:x1, y0:y1] != id, cell['position_PD__binned_12.5'])
        # if id > 500:
        #     break


        dist = 50
        cnt_labels = np.unique(labels[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)]).size - 1
        cnt_pixels = tissue_map[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)].sum()
        rho = cnt_labels / cnt_pixels / res_x / res_y
        objects.loc[id, 'HC_density__50'] = rho

        dist = 100
        cnt_labels = np.unique(labels[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)]).size - 1
        cnt_pixels = tissue_map[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)].sum()
        rho = cnt_labels / cnt_pixels / res_x / res_y
        objects.loc[id, 'HC_density__100'] = rho

        density_map[x0:x1, y0:y1] = density_map[x0:x1, y0:y1].where(
            labels[x0:x1, y0:y1] != id, rho)
        
        dist = 200
        cnt_labels = np.unique(labels[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)]).size - 1
        cnt_pixels = tissue_map[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)].sum()
        rho = cnt_labels / cnt_pixels / res_x / res_y
        objects.loc[id, 'HC_density__200'] = rho

        dist = 500
        cnt_labels = np.unique(labels[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)]).size - 1
        cnt_pixels = tissue_map[max(0, x - dist):min(Lx, x + dist + 1), max(0, y - dist):min(Ly, y + dist + 1)].sum()
        rho = cnt_labels / cnt_pixels / res_x / res_y
        objects.loc[id, 'HC_density__500'] = rho

    results = [
        prepare_save_tif(original, normalise=True),
        prepare_save_tif(area_map, normalise=False),
        prepare_save_tif(hexatic_map * np.iinfo(np.short).max, normalise=False),
        prepare_save_tif(hexatic_map_c * np.iinfo(np.short).max, normalise=False),
        prepare_save_tif(neighbour_map, normalise=True),
        prepare_save_tif(voronoi.where(voronoi==0, voronoi%20+1) / 21 * np.iinfo(np.short).max, normalise=False),
        prepare_save_tif(tissue_map * np.iinfo(np.short).max, normalise=False),
        prepare_save_tif(density_map, normalise=True),
    ]
    save_image(
        xr.concat(results, dim='c', coords='minimal').assign_coords(c=range(len(results))),
        normalise=False,
        base_dir=None,
        out_dir=out_dir,
        filename=filename_raw+'__results.tif'
    )

    objects.to_csv(os.path.join(out_dir, f"{filename_raw}__final.csv"))

    

area_map = None
del area_map
hexatic_map = None
del hexatic_map
hexatic_map_c = None
del hexatic_map_c
neighbour_map = None
del neighbour_map
tissue_map = None
del tissue_map
results = None
del results

In [ ]:
print(objects.to_string())

In [ ]:
raise RuntimeError("Please proceed with analyse_restore_whole_BP.ipynb script to merge across files.")

In [ ]:
Objects['Stage'] = Objects['filename'].apply(lambda name: name.split('/')[0])
Objects['Stage'] = pd.Categorical(Objects['Stage'], ["E8", "E10", "E12"])

Objects.to_csv(os.path.join(basepath, path, f"summary.csv"))

In [ ]:
print(Objects[['filename', 'curvature_tangent_orientation']].to_string())

In [ ]:
print(Objects['barycenter_x'].mean())
print(Objects['curvature_center_x'].mean())
print(Objects['barycenter_y'].mean())
print(Objects['curvature_center_y'].mean())

# Eval cell properties

In [ ]:
HC_number = Objects.groupby(by=['filename'])['area'].count().reset_index()
HC_number = HC_number.rename(columns={'area': 'HC number'})
HC_number['Stage'] = HC_number['filename'].apply(lambda name: name.split('/')[0])
HC_number['Stage'] = pd.Categorical(HC_number['Stage'], ["E8", "E10", "E12"])

HC_number = HC_number.loc[HC_number['HC number'] > 0]

HC_number = HC_number.sort_values(by='Stage')
print(HC_number)

fig, ax = plt.subplots(1, 1, figsize=[1.5, 1.5])
sns.swarmplot(data=HC_number, x='Stage', y='HC number', color='red')

ax.set_ylim([0, 15000])
ax.set_yticks([0, 5000, 10000, 15000])

ax.set_xlim([-0.5, 3.5])
ax.set_xticks(range(4))
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'])

plt.savefig(f"{os.path.join(basepath, path)}/HC_number.svg")




fig, ax = plt.subplots(1, 1, figsize=[1.5, 1.5])

HC_number = Objects.groupby(by=['filename', 'position_PD__binned_20'])['area'].count().reset_index()
HC_number = HC_number.rename(columns={'area': 'HC number'})
HC_number['Stage'] = HC_number['filename'].apply(lambda name: name.split('/')[0])
HC_number['Stage'] = pd.Categorical(HC_number['Stage'], ["E8", "E10", "E12"])

HC_number = HC_number.loc[HC_number['HC number'] > 0]

HC_number = HC_number.sort_values(by='Stage')
print(HC_number)

sns.swarmplot(data=HC_number, x='Stage', y='HC number', hue='position_PD__binned_20', palette='autumn', alpha=0.7)

ax.set_xlim([-0.5, 3.5])
ax.set_xticks(range(4))
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'])

ax.set_ylim([0, 3500])
ax.set_yticks([0, 1000, 2000, 3000])

plt.savefig(f"{os.path.join(basepath, path)}/HC_number__binned.svg")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[3.5, 3.5])

HC_number = Objects.groupby(by=['filename', 'position_PD__binned_20'])['area'].count().reset_index()
HC_number = HC_number.rename(columns={'area': 'HC number'})
HC_number['Stage'] = HC_number['filename'].apply(lambda name: name.split('/')[0])
HC_number['Stage'] = pd.Categorical(HC_number['Stage'], ["E8", "E10", "E12"])

HC_number = HC_number.loc[HC_number['HC number'] > 0]

HC_number = HC_number.sort_values(by='Stage')
print(HC_number)

sns.pointplot(data=HC_number, x='Stage', y='HC number', hue='position_PD__binned_20', palette='autumn')

ax.set_xlim([-0.5, 3.5])
ax.set_xticks(range(4))
ax.set_xticklabels(['E8', 'E10', 'E12', 'E14'])

ax.set_ylim([0, 3500])
ax.set_yticks([0, 1000, 2000, 3000])

plt.savefig(f"{os.path.join(basepath, path)}/HC_number__binned_mean.svg")

In [ ]:
for filename in filenames:
    print(Objects.loc[Objects['filename']==filename])
    fig, ax = plt.subplots(1, 1, figsize=[15, 6])
    ax.invert_yaxis()
    ax.set_aspect(1)
    sns.scatterplot(
        data=Objects.loc[Objects['filename']==filename],
        x='barycenter_x',
        y='barycenter_y',
        hue='position_PD__binned_20',
        palette='seismic',
        s=0.05, linewidth=0, 
    )

In [ ]:
fig, ax = plt.subplots(1,1)

# Plot cell properties

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.violinplot(data=Objects, x='area')

In [ ]:
plot = dict({
    # 'area': True,
    # 'area_log': True,
    # 'hexatic': True,
    # 'neighbour': True,
    'id': True,
    # 'elongation': True,
    # 'elongation_orientation': True,
    # 'HCA': True,
    # 'HCA_polarity': True,
    # 'HCA_polarity_vs_elongation_orientation': True,
})

print(filenames)
for filename in filenames:
    print(filename)

    _labels = Labels[filename]
    _image = Images[filename]
    if 'z' in _image.dims:
        _image = _image.max(dim='z')
    _image.attrs = Images[filename].attrs
    _voronoi = Voronoi[filename]

    res_x = _image.attrs['resolution_x']
    res_y = _image.attrs['resolution_y']
    aspect = res_x / res_y
    extent=[0, _image.shape[0]/res_x, 0, _image.shape[1]/res_y]
    print(f"{_image.attrs['resolution_scale']}: {res_x} x {res_y} ({aspect})")
    if not np.isfinite(aspect):
        raise
    
    if not Objects.loc[Objects['filename'] == filename].empty:
        objects = Objects.loc[Objects['filename'] == filename].sort_values(by='id').set_index('id').reset_index()
        objects = objects.loc[~np.asarray(objects['exclude'], dtype=bool)]

        print(objects.to_string())

    if True:
        _area_map = xr.zeros_like(_labels, dtype=float)
        _hexatic_map = xr.zeros_like(_labels, dtype=float)
        _neighbour_map = xr.zeros_like(_labels, dtype=float)
        _id_map = xr.zeros_like(_labels, dtype=float)
        _elongation_map = xr.zeros_like(_labels, dtype=float)
        _orientation_map = xr.zeros_like(_labels, dtype=float)
        _HCA_map = xr.zeros_like(_labels, dtype=float)
        _HCA_polarity = xr.zeros_like(_labels, dtype=float)
        _HCA_polarity_amplitude = xr.zeros_like(_labels, dtype=float)
        _HCA_polarity_vs_elongation_orientation = xr.zeros_like(_labels, dtype=float)


        isExist = os.path.exists(os.path.join(basepath, path, filename[:-4]))
        if not isExist:
            os.makedirs(os.path.join(basepath, path, filename[:-4]))

        isExist = os.path.exists(os.path.join(basepath, path, filename[:-4], "area"))
        if not isExist:
            os.makedirs(os.path.join(basepath, path, filename[:-4], "area"))

        isExist = os.path.exists(os.path.join(basepath, path, filename[:-4], "hexatic_order"))
        if not isExist:
            os.makedirs(os.path.join(basepath, path, filename[:-4], "hexatic_order"))

        isExist = os.path.exists(os.path.join(basepath, path, filename[:-4], "neighbours"))
        if not isExist:
            os.makedirs(os.path.join(basepath, path, filename[:-4], "neighbours"))

        i = 0
        step_x = 500
        step_y = 250
        for x in range(0, _image.shape[0], step_x):
            for y in range(0, _image.shape[1], step_y):
                i += 1
                
                image = _image.isel(
                    x=slice(x, np.min([x+step_x, _image.shape[0]])),
                    y=slice(y, np.min([y+step_y, _image.shape[1]]))
                )
                labels = _labels.isel(
                    x=slice(x, np.min([x+step_x, _image.shape[0]])),
                    y=slice(y, np.min([y+step_y, _image.shape[1]]))
                )

                area_map = xr.zeros_like(labels) * np.nan
                hexatic_map = xr.zeros_like(labels) * np.nan
                neighbour_map = xr.zeros_like(labels) * np.nan
                id_map = xr.zeros_like(labels) * np.nan
                elongation_map = xr.zeros_like(labels) * np.nan
                orientation_map = xr.zeros_like(labels) * np.nan
                HCA_map = xr.zeros_like(labels) * np.nan
                HCA_polarity = xr.zeros_like(labels) * np.nan
                HCA_polarity_amplitude = xr.zeros_like(labels, dtype=float)
                HCA_polarity_vs_elongation_orientation = xr.zeros_like(labels, dtype=float) * np.nan

                for id in np.unique(labels):
                    if not objects.loc[objects['id']==id].empty:
                        object = objects.loc[objects['id']==id].iloc[0]
                        if plot.get('area', False) or plot.get('area_log', False):
                            area_map = area_map.where(labels != id, object['area'])
                        if plot.get('hexatic', False):
                            hexatic_map = hexatic_map.where(labels != id, object['hexatic_order'])
                        if plot.get('neighbour', False):
                            neighbour_map = neighbour_map.where(labels != id, object['num_neighbors_voronoi'])
                        if plot.get('id', False):
                            id_map = id_map.where(labels != id, id%20+1)
                        if plot.get('elongation', False):
                            elongation_map = elongation_map.where(labels != id, object['elongation'])
                        if plot.get('elongation_orientation', False):
                            orientation_map = orientation_map.where(labels != id, object['elongation_orientation'] - object['curvature_tangent_orientation'])
                        if plot.get('HCA', False):
                            HCA_map = HCA_map.where(labels != id, image / image.where(labels == id).max())
                        if plot.get('HCA_polarity', False):
                            HCA_polarity = HCA_polarity.where(labels != id, object['HCA_polarity_orientation'] - object['curvature_tangent_orientation'])
                            HCA_polarity_amplitude = HCA_polarity_amplitude.where(labels != id, object['HCA_polarity'])
                        if plot.get('HCA_polarity_vs_elongation_orientation', False):
                            val = np.mod(object['HCA_polarity_orientation'] - object['elongation_orientation'] + math.pi, math.pi)
                            HCA_polarity_vs_elongation_orientation = HCA_polarity_vs_elongation_orientation.where(labels != id, val)
                            

                _area_map.data[x:x+step_x, y:y+step_y] += area_map.data
                _hexatic_map.data[x:x+step_x, y:y+step_y] += hexatic_map.data
                _neighbour_map.data[x:x+step_x, y:y+step_y] += neighbour_map.data
                _id_map.data[x:x+step_x, y:y+step_y] += id_map.data
                _elongation_map.data[x:x+step_x, y:y+step_y] += elongation_map.data
                _orientation_map.data[x:x+step_x, y:y+step_y] += orientation_map.data
                _HCA_map.data[x:x+step_x, y:y+step_y] += HCA_map.data
                _HCA_polarity.data[x:x+step_x, y:y+step_y] += HCA_polarity.data
                _HCA_polarity_amplitude.data[x:x+step_x, y:y+step_y] += HCA_polarity_amplitude.data
                _HCA_polarity_vs_elongation_orientation.data[x:x+step_x, y:y+step_y] += HCA_polarity_vs_elongation_orientation.data

        _orientation_map = np.mod(_orientation_map + math.pi, math.pi) - math.pi/2.
        _HCA_polarity = np.mod(_HCA_polarity + 2 * math.pi, 2 * math.pi) - math.pi
        _HCA_polarity_vs_elongation_orientation = math.pi / 2. - np.abs(math.pi/2. - np.absolute(_HCA_polarity_vs_elongation_orientation))

    figsize = _image.shape[0] / _image.shape[1]

    if plot.get('area', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_area_map.T, interpolation='none', cmap='RdYlBu', alpha=0.65, vmin=0, vmax=objects['area'].max(), extent=extent)
        fig.colorbar(im, ax=ax, label=f'Apical area [$\mu m^2$]')

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"area.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"area.png"), dpi=600)
        plt.close()

    if plot.get('area_log', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(
            _area_map.T, 
            interpolation='none', 
            cmap='gist_rainbow', 
            alpha=0.65, 
            norm=mpl.colors.LogNorm(vmin=4, vmax=40),
            extent=extent
        )

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        fig.colorbar(im, ax=ax, label=f'Apical area [$\mu m^2$]', ticks=[5, 10, 20, 40])
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"area_log.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"area_log.png"), dpi=600)
        plt.close()


    if plot.get('hexatic', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_hexatic_map.T, interpolation='none', cmap='seismic', vmin=0, vmax=0.8, alpha=0.65, extent=extent)
        fig.colorbar(im, ax=ax, label='Hexatic order')


        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"hexatic_order.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"hexatic_order.png"), dpi=600)
        plt.close()


    if plot.get('neighbour', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_neighbour_map.T, interpolation='none', cmap='seismic', vmin=3, vmax=9, alpha=0.65, extent=extent)
        fig.colorbar(im, ax=ax, label='Neighbour number')


        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"neighbours.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"neighbours.png"), dpi=600)
        plt.close()


    if plot.get('id', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_id_map.where(_id_map > 0).T, interpolation='none', cmap='tab20', alpha=0.85, extent=extent)
        fig.colorbar(im, ax=ax, label='Segmented')


        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"identified.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"identified.png"), dpi=600)
        plt.close()


    if plot.get('elongation', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_elongation_map.T, interpolation='none', cmap='Reds', alpha=0.85, vmin=0, extent=extent)
        fig.colorbar(im, ax=ax, label='Elongation')

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"elongation.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"elongation.png"), dpi=600)
        plt.close()


    if plot.get('elongation_orientation', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_orientation_map.T, interpolation='none', cmap='hsv', alpha=0.85, vmin=-math.pi/2., vmax=math.pi/2., extent=extent)
        cbar = fig.colorbar(im, ax=ax, label='Orientation of elongation', ticks=[-math.pi/2., 0,  math.pi/2.])
        cbar.ax.set_yticklabels(['Proximally - Distally', 'Inferiorly - Superiorly', 'Proximally - Distally'])

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"elongation_orientation.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"elongation_orientation.png"), dpi=600)
        plt.close()


    if plot.get('HCA', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        scale = (objects['area_pixels']/math.pi)**0.5

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_HCA_map.T, interpolation='none', cmap='inferno', vmin=0, vmax=1., extent=extent)
        fig.colorbar(im, ax=ax, label='local HCA profile')

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')
        ax.quiver(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'],
                scale * objects['HCA_polarity_px'], scale * objects['HCA_polarity_py'],
                angles='xy', scale_units='xy', scale=1./_image.attrs['resolution_x']**2, 
                color='green', width=0.00033, headwidth=1, headlength=1
                )

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_profile.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_profile.png"), dpi=600)
        plt.close()


    if plot.get('HCA_polarity', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        fig.tight_layout()
        
        scale = (objects['area_pixels']/math.pi)**0.5

        _HCA_polarity_amplitude = (_HCA_polarity_amplitude - _HCA_polarity_amplitude.min()) / (_HCA_polarity_amplitude.max() - _HCA_polarity_amplitude.min())

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_HCA_polarity.T, interpolation='none', cmap='hsv', vmin=-math.pi, vmax=math.pi, alpha=_HCA_polarity_amplitude.T, extent=extent)
        cbar = fig.colorbar(im, ax=ax, label='HCA orientation', ticks=[-math.pi, -math.pi/2., 0,  math.pi/2., math.pi])
        cbar.ax.set_yticklabels(['Distally', 'Inferiorly', 'Proximally', 'Superiorly', 'Distally'])

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')

        ax.quiver(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'],
                scale * objects['HCA_polarity_px'], scale * objects['HCA_polarity_py'],
                angles='xy', scale_units='xy', scale=1./_image.attrs['resolution_x']**2, 
                color='white', width=0.00033, headwidth=1, headlength=1
                )

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity.png"), dpi=600)
        plt.close()


        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        
        scale = (objects['area_pixels']/math.pi)**0.5

        _HCA_polarity_amplitude = (_HCA_polarity_amplitude - _HCA_polarity_amplitude.min()) / (_HCA_polarity_amplitude.max() - _HCA_polarity_amplitude.min())

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_HCA_polarity.T, interpolation='none', cmap='hsv', vmin=-math.pi, vmax=math.pi, extent=extent)
        cbar = fig.colorbar(im, ax=ax, label='HCA orientation', ticks=[-math.pi, -math.pi/2., 0,  math.pi/2., math.pi])
        cbar.ax.set_yticklabels(['Distally', 'Inferiorly', 'Proximally', 'Superiorly', 'Distally'])

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')

        ax.quiver(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'],
                scale * objects['HCA_polarity_px'], scale * objects['HCA_polarity_py'],
                angles='xy', scale_units='xy', scale=1./_image.attrs['resolution_x']**2, 
                color='white', width=0.00033, headwidth=1, headlength=1
                )

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity__.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity__.png"), dpi=600)
        plt.close()

    if plot.get('HCA_polarity_vs_elongation_orientation', False):
        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(aspect)
        

        _HCA_polarity_amplitude = (_HCA_polarity_amplitude - _HCA_polarity_amplitude.min()) / (_HCA_polarity_amplitude.max() - _HCA_polarity_amplitude.min())

        ax.imshow(_image.T, interpolation='none', cmap='gray', extent=extent)
        im = ax.imshow(_HCA_polarity_vs_elongation_orientation.T, interpolation='none', cmap='Reds', extent=extent) # , vmin=0, vmax=math.pi)
        cbar = fig.colorbar(im, ax=ax, label='Diff. HCA polarity vs elongation')

        ax.set_xlabel('P-D axis [$\mu m$]')
        ax.set_ylabel('S-I axis [$\mu m$]')

        scale = objects['elongation'] * (objects['area_pixels']/math.pi)**0.5
        ax.quiver(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'],
                scale * np.cos(objects['elongation_orientation']), scale * np.sin(objects['elongation_orientation']),
                angles='xy', scale_units='xy', scale=1./_image.attrs['resolution_x']**2, pivot='mid',
                color='black', width=0.00033, headwidth=1, headlength=0
                )
        scale = (objects['area_pixels']/math.pi)**0.5
        ax.quiver(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'],
                scale * objects['HCA_polarity_px'], scale * objects['HCA_polarity_py'],
                angles='xy', scale_units='xy', scale=1./_image.attrs['resolution_x']**2, 
                color='magenta', width=0.00033, headwidth=1, headlength=1
                )

        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity_vs_elongation_orientation.svg"),format='svg')
        fig.savefig(os.path.join(basepath, path, filename[:-4], f"HCA_polarity_vs_elongation_orientation.png"), dpi=600)
        plt.close()

_labels=None
del _labels
_image=None
del _image
_voronoi=None
del _voronoi
objects=None
del objects
_area_map = None
del _area_map
_hexatic_map = None
del _hexatic_map
_neighbour_map = None
del _neighbour_map
_id_map = None
del _id_map
_elongation_map = None
del _elongation_map
_orientation_map = None
del _orientation_map
_HCA_map=None
del _HCA_map
_HCA_polarity=None
del _HCA_polarity
_HCA_polarity_amplitude=None
del _HCA_polarity_amplitude
area_map = None
del area_map
hexatic_map = None
del hexatic_map
neighbour_map = None
del neighbour_map
id_map = None
del id_map
elongation_map = None
del elongation_map
orientation_map = None
del orientation_map
HCA_map=None
del HCA_map
HCA_polarity=None
del HCA_polarity
HCA_polarity_amplitude=None
del HCA_polarity_amplitude

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.histplot(Objects, x='HCA_polarity')
Objects['HCA_polarity'].mean()

In [ ]:
raise

# WIP

In [ ]:
thresh = skimage.filters.threshold_otsu(img)
print(thresh)

bw = np.where(img > thresh, img, 0)
bw = skimage.morphology.binary_dilation(bw)
# bw = skimage.morphology.binary_erosion(bw)
print(bw)

fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(bw.T, cmap='gray')


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(img.flatten(), bins=2000)

In [ ]:
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background

bw = bw > 0

distance = scipy.ndimage.distance_transform_edt(bw)
coords = skimage.feature.peak_local_max(distance, labels=bw, min_distance=5)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = scipy.ndimage.label(mask)
labels = skimage.segmentation.watershed(-distance, markers, mask=bw)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[25, 10])
ax1.set_aspect(1)
ax2.set_aspect(1)
ax3.set_aspect(1)
ax1.imshow(img_.T, cmap='gray')
ax3.imshow(distance.T, cmap='gray')
ax1.scatter(coords[:, 0], coords[:, 1], s=1)
ax2.scatter(coords[:, 0], coords[:, 1], s=1)
ax3.scatter(coords[:, 0], coords[:, 1], s=1)
# ax2.invert_yaxis()

ax2.imshow(np.where(labels > 0, labels, np.nan).T)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(img_.where(labels > 0).T, cmap='gray')



In [ ]:
# # analyse the objects
# objects = pd.DataFrame()
# for id in np.unique(labels):
#     if id == 0: 
#         continue
    
#     # the coordinate axes
#     argwhere = np.argwhere((labels == id).data)
#     argwhere = np.asarray(argwhere, dtype=float)
    
#     # calculate in pixel dimensions
#     barycenter = argwhere.mean(0)
#     max = argwhere.max(0)
#     min = argwhere.min(0)

#     if min[0] == 0 or min[1] == 0 or max[0] >= len(img_.x)-1 or max[1] >= len(img_.y)-1:
#         continue

#     mask = np.where(labels == id, 1, 0)
#     volume = mask.sum()

#     objects.loc[id, 'volume_pixels'] = volume
#     for i, axis in enumerate(['x', 'y']):
#         objects.loc[id, 'barycenter_{}_pixels'.format(axis)] = barycenter[i]
#     for i, axis in enumerate(['x', 'y']):
#         objects.loc[id, 'min_{}_pixels'.format(axis)] = min[i]
#         objects.loc[id, 'max_{}_pixels'.format(axis)] = max[i]


# # use image resolution
# res_x = img_.attrs['resolution_x']
# res_y = img_.attrs['resolution_y']
# res_z = img_.attrs['resolution_z']

# objects['volume'] = objects['volume_pixels'] / res_x / res_y

# objects['barycenter_x'] = objects['barycenter_x_pixels'] / res_x
# objects['barycenter_y'] = objects['barycenter_y_pixels'] / res_y

# objects['min_x'] = objects['min_x_pixels'] / res_x
# objects['max_x'] = objects['max_x_pixels'] / res_x
# objects['min_y'] = objects['min_y_pixels'] / res_y
# objects['max_y'] = objects['max_y_pixels'] / res_y

# print(objects)

In [ ]:
i = 0
for x in range(0, img.shape[0], 500):
    for y in range(0, img.shape[1], 250):
        if img[x:x+500, y:y+250].mean() < 0.005:
            continue

        fig, ax = plt.subplots(1, 1, figsize=[20, 20 / figsize])
        ax.set_aspect(1)
        ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
        # ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

        # ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
        ax.scatter(coords[:, 0], coords[:, 1])

        ax.set_xlim(x, x+500)
        ax.set_ylim(y, y+250)


        fig.savefig(os.path.join(basepath, path, filenames[0][:-4], f"crop_{i}.png"))
        i += 1
        plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(200, 400)
ax.set_ylim(2300, 2500)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(1000, 1200)
ax.set_ylim(1900, 2100)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(2000, 2200)
ax.set_ylim(1600, 1800)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(3000, 3200)
ax.set_ylim(1400, 1600)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(4000, 4200)
ax.set_ylim(1100, 1300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[10, 10])
ax.set_aspect(1)
ax.imshow(np.where(labels > 0, 1, 0.5).T * img_.T, cmap='gray')
# ax.imshow(np.where(labels > 0, 1, 0).T, cmap='gray')

# ax.scatter(objects['barycenter_x_pixels'], objects['barycenter_y_pixels'], s=1)
ax.scatter(coords[:, 0], coords[:, 1])

ax.set_xlim(5000, 5200)
ax.set_ylim(1300, 1600)

In [ ]:
N = 64
print(N*12)
x = np.linspace(0, N, 500)

l = 4
r = 0.5
def A(x):
    # return (l - r) * np.cos(math.pi/2 * x / N) + r
    return (l - r) * np.cos(math.pi /2 * x/N)**2 + r

def radius(x):
    return (A(x) / math.pi)**0.5

fig, axes = plt.subplots(2, 1, sharex=True, figsize=[4, 4])
fig.tight_layout()

axes[0].plot(x, A(x))
axes[0].plot(x, A(x-2*radius(x)), ls=':')
axes[0].plot(x, A(x+2*radius(x)), ls=':')
axes[0].legend()

axes[1].plot(x, (A(x-2*radius(x)) - A(x+2*radius(x))))
axes[1].plot(x, (A(x-2*radius(x)) - A(x+2*radius(x)))/A(x), label='relative')
axes[1].legend()

axes[0].set_ylabel("Area")
axes[1].set_ylabel("Area difference")

axes[0].set_xlim(0, N)
axes[0].set_ylim(0, 4)
axes[1].set_ylim(0, None)
